In [ ]:
import sys, traceback
import cv2
import numpy as np
import argparse
import string
from plantcv import plantcv as pcv

In [ ]:
# Set debug mode
pcv.params.debug=args.debug 
# Set output directory
pcv.params.debug_outdir=args.outdir 

# Read image (converting fmax and track to 8 bit just to create a mask, use 16-bit for all the math)
mask, path, filename = pcv.readimage(args.fmax)
# mask = cv2.imread(args.fmax)
track = cv2.imread(args.track)

mask1, mask2, mask3 = cv2.split(mask)

In [ ]:
# Mask pesky track autofluor
track1 = pcv.rgb2gray_hsv(track, 'v')
track_thresh = pcv.threshold.binary(track1, 0, 255, 'light')
track_inv = pcv.invert(track_thresh)
track_masked = pcv.apply_mask(mask1, track_inv, 'black')

In [ ]:
# Threshold the image
fmax_thresh = pcv.threshold.binary(track_masked, 20, 255, 'light')

In [ ]:
# Median Filter
s_mblur = pcv.median_blur(fmax_thresh, 5)
s_cnt = pcv.median_blur(fmax_thresh, 5)

In [ ]:
# Fill small objects
s_fill = pcv.fill(s_mblur, 110)
sfill_cnt = pcv.fill(s_mblur, 110)

In [ ]:
# Identify objects
id_objects,obj_hierarchy = pcv.find_objects(mask, sfill_cnt)

In [ ]:
# Define ROI
roi1, roi_hierarchy = pcv.roi.rectangle(x=100, y=100, h=200, w=200, img=mask)

In [ ]:
# Decide which objects to keep
roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(mask, 'partial', roi1, roi_hierarchy, id_objects, obj_hierarchy)

In [ ]:
# Object combine kept objects
obj, masked = pcv.object_composition(mask, roi_objects, hierarchy3)

In [ ]:
################ Analysis ################  
    
outfile=False
if args.writeimg==True:
    outfile=args.outdir+"/"+filename

# Find shape properties, output shape image (optional)
shape_header, shape_data, shape_img = pcv.analyze_object(mask, obj, masked, args.outdir + '/' + filename)

# Fluorescence Measurement (read in 16-bit images)
fdark = cv2.imread(args.fdark, -1)
fmin = cv2.imread(args.fmin, -1)
fmax = cv2.imread(args.fmax, -1)

fvfm_header, fvfm_data, fvfm_images = pcv.fluor_fvfm(fdark,fmin,fmax,kept_mask, args.outdir+'/'+filename, 1000)

# Write shape and nir data to results file
result=open(args.result,"a")
result.write('\t'.join(map(str,shape_header)))
result.write("\n")
result.write('\t'.join(map(str,shape_data)))
result.write("\n")
for row in shape_img:  
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.write('\t'.join(map(str,fvfm_header)))
result.write("\n")
result.write('\t'.join(map(str,fvfm_data)))
result.write("\n")
result.close()